This file intends to extract all the information from the input sheet and store it in a dictionary. The keys would be file name cell name, and the values would be the func name and the arguments, where the arguments are {arg_name: value}

In [1]:
import pandas as pd
import os
import numpy as np
import json
from collections import Counter
from collections import OrderedDict

In [2]:
# read in the data
def load_data(path):
    """ read in excel sheets
        grab all .xlsx files in the dir specified by path """
    files_under_path = os.listdir(path)
    fnames = [f[:-5] for f in files_under_path if f.endswith('.xlsx')]

    all_files = {}
    for i in range(len(fnames)):
        file = pd.read_excel(f'{path}/{fnames[i]}.xlsx', sheet_name=None, engine='openpyxl')
        all_files[fnames[i]] = file

    return fnames, all_files

In [3]:
os.getcwd()

'c:\\Users\\yihuaqia\\work\\markdraw_stat\\source_code\\dev_test\\analysis'

In [4]:
fnames, all_files = load_data('../../input_sheets/1227')

In [5]:
all_files.keys()

dict_keys(['canon_1227_d3t4_jm2_siv_2x', 'canon_1227_d3t7_jm5_siv_2x', 'canon_1227_d3t7_psv', 'canon_1227_gmb_siv_2x', 'canon_1227_gms_siv_2x', 'canon_lunarlake_psm_final_1'])

In [6]:
fnames  # list of file names

['canon_1227_d3t4_jm2_siv_2x',
 'canon_1227_d3t7_jm5_siv_2x',
 'canon_1227_d3t7_psv',
 'canon_1227_gmb_siv_2x',
 'canon_1227_gms_siv_2x',
 'canon_lunarlake_psm_final_1']

In [7]:
all_files[fnames[1]].keys()  # sheet names

dict_keys(['canon', 'all_functions', 'all_marks', 'parents', 'review_parent', 'dummy_beard', 'dummy_filler', 'fdr_dummy', 'kga', 'diag_dummy', 'zonal_bkg', 'xy4', 'pound', 'hatch', 'swirl_xy4', 'swirl_cross', 'swirl_hatch', 'marklist', 'delivery'])

In [ ]:
def get_cell_info2(file):
    """ get the cell names from the StartLayoutAssembler or zonal_background """
    # df_list = []
    od_df = OrderedDict()
    name_list = []
    for sname in file.keys():
        cell_names = []
        new_cell_idx = []
        df = pd.DataFrame(file[sname])
        # check if the StartLayoutAssembler is in any place in the df
        if 'StartLayoutAssembler' not in df.values and 'zonal_background' not in df.values:
            continue
        
        # get cell names; i is the row index
        for i in range(df.shape[0]):
            # print(f'{i}: {df.iloc[i,1]}')
            if df.iloc[i,1] == 'StartLayoutAssembler' or df.iloc[i,1] == 'zonal_background':
                c_name = df.iloc[i]['CellName.string']

                if c_name not in cell_names:
                    # cell_names.append((c_name,i))
                    cell_names.append(c_name)
                    new_cell_idx.append(i)

        # print(f'{fname}: {cell_names}')
        # slice the cells based on cellnames
        df_cells = {} 
        for i in range(len(cell_names)-1):
            df_cells[cell_names[i]] = df.iloc[new_cell_idx[i]:new_cell_idx[i+1],:].dropna(how='all')
        #     df_cells[cell_names[i][0]] = df.iloc[cell_names[i][1]:cell_names[i+1][1],:].dropna(how='all')
        # df_cells[cell_names[-1][0]] = df.iloc[cell_names[-1][1]:,:].dropna(how='all')
        df_cells[cell_names[-1]] = df.iloc[new_cell_idx[-1]:,:].dropna(how='all')
        
        # df_list.append(df_cells)
        od_df[sname] = df_cells
        name_list.append(cell_names)
    return od_df, name_list



In [9]:
tmp_file_name = 'canon_1227_d3t4_jm2_siv_2x'
od_df, name_lst = get_cell_info2(all_files[tmp_file_name])

In [10]:
od_df.keys()

odict_keys(['canon', 'all_marks', 'parents', 'review_parent', 'dummy_beard', 'dummy_filler', 'fdr_dummy', 'kga', 'diag_dummy', 'zonal_bkg', 'xy4', 'pound', 'hatch', 'swirl_xy4', 'swirl_cross', 'swirl_hatch'])

In [11]:
od_df['all_marks']['122700c_d3t4_siv374d']

,EXECUTE,SampleStartLayoutAssembler,Library.string,CellName.string,OpenCellViewMode.string,CellSizeX.float,CellSizeY.float,CenterX.float,CenterY.float,Category.string
69,y,zonal_background,canon27d3t4lib2_p27_lay,122700c_d3t4_siv374d,(61.74 63.72),or1.tccDebug,1,2,True,NaN
70,y,StartLayoutAssembler,canon27d3t4lib2_p27_lay,122700c_d3t4_siv374d,a,61.74,63.72,0,0,NaN
71,y,create_rectangle,cv,or2.tccDebug,-24 -24 24 24,NaN,NaN,NaN,NaN,NaN
72,y,chopped_tvpa_canon,cv,JM2_mask.drawing,36,2,9,0.5,NaN,NaN
73,y,chopped_tvpa_canon,cv,SIV_mask.drawing,36,2,9,0.5,NaN,NaN
74,y,shape_size,cv,SIV_mask.drawing,-0.5,-0.5,-0.5,-0.5,0,NaN
75,y,bool_and_not,cv,or1.tccDebug,or2.tccDebug,JM2_mask.drawing,NaN,NaN,NaN,NaN


In [12]:
name_lst[1]

['122700c_d3t4_siv_swirl',
 '122700c_d3t4_jm2_dummy_3_1',
 '122700c_d3t4_siv022d',
 '122700c_d3t4_siv023d',
 '122700c_d3t4_siv024d',
 '122700c_d3t4_siv078d',
 '122700c_d3t4_siv173d',
 '122700c_d3t4_siv222d',
 '122700c_d3t4_siv223d',
 '122700c_d3t4_siv224d',
 '122700c_d3t4_siv355d',
 '122700c_d3t4_siv374d']

In [14]:
od_df.keys()

odict_keys(['canon', 'all_marks', 'parents', 'review_parent', 'dummy_beard', 'dummy_filler', 'fdr_dummy', 'kga', 'diag_dummy', 'zonal_bkg', 'xy4', 'pound', 'hatch', 'swirl_xy4', 'swirl_cross', 'swirl_hatch'])

In [19]:
sheet_name = list(od_df.keys())[1]  # the second sheet name
cell_name = name_lst[1][3]    # the fourth cell name in the second sheet
od_df[sheet_name][cell_name]

,EXECUTE,SampleStartLayoutAssembler,Library.string,CellName.string,OpenCellViewMode.string,CellSizeX.float,CellSizeY.float,CenterX.float,CenterY.float,Category.string
23,y,StartLayoutAssembler,canon27d3t4lib2_p27_lay,122700c_d3t4_siv023d,w,61.74,63.72,0,0,NaN
24,y,xy_canon,cv,SIV_mask.drawing,(1.8 1.8),(25 25),(28.2 28.2),((5)),NaN,NaN
25,y,create_instance,cv,canon27d3t4lib1_p27_lay,122700c_d3t4_jm2_dummy_3_1,NaN,NaN,NaN,NaN,NaN


In [22]:
def get_df_info(technology,file_name, sheet_name, cell_name):
    """ get the cell information from the file """
    in_file_path = f'../../input_sheets/{technology}'
    fnames, all_files = load_data(in_file_path)
    od_df, name_lst = get_cell_info2(all_files[file_name])
    return od_df[sheet_name][cell_name]

In [24]:
print(get_df_info(1222,'canon_1222_d4t1t5_S22Y_GMB_GV1_2x', 'xy4', '122200c_d4t1t5_s22y_gmb022d'))

   EXECUTE SampleStartLayoutAssembler        Library.string  \
26       y       StartLayoutAssembler  frmcanon2d4_x22a_lay   
27       y                   xy_canon                    cv   

                CellName.string OpenCellViewMode.string CellSizeX.float  \
26  122200c_d4t1t5_s22y_gmb022d                       w           61.74   
27                  gmb.drawing               (1.6 1.6)         (25 25)   

   CellSizeY.float CenterX.float CenterY.float Category.string Unnamed: 10  \
26           63.72             0             0             NaN         NaN   
27     (28.4 28.4)         ((5))           NaN             NaN         NaN   

   Unnamed: 11 Unnamed: 12  
26         NaN         NaN  
27         NaN         NaN  
